In [1]:
import pandas as pd
import time
import math
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import KFold
import statistics

In [2]:
def read_in_tables(n=78):
    df_list = {}
    for i in range(1, n):

        table1_name = 'test_df' + str(i)
        table2_name = 'train_df' + str(i)

        df = pd.read_csv(f'../datasets/training/SARIMA/by_area/ml_{i}.csv', parse_dates=['date'])

        globals()[table1_name] = df[(df.date <= pd.to_datetime('2019-12-31')) & (df.date >= pd.to_datetime('2019-01-01'))]
        globals()[table2_name] = df[(df.date < pd.to_datetime('2019-01-01')) & (df.date > pd.to_datetime('2016-01-01'))]

        globals()[table1_name].fillna(0, inplace=True)
        globals()[table2_name].fillna(0, inplace=True)

        globals()[table1_name].set_index('date', inplace=True)
        globals()[table2_name].set_index('date', inplace=True)

        df_list[table1_name] = globals()[table1_name]
        df_list[table2_name] = globals()[table2_name]

    return df_list

In [3]:
class LinearRegression:
    # Class-level attributes for model coefficients and parameters
    m = []  # Model coefficients for features
    b = 0   # Model intercept
    initial_learning_rate = 0.01  # Initial learning rate (how fast the model moves the coefficients)
    learning_rate = initial_learning_rate
    learning_rate_decay = 0.99    # Learning rate decay factor (over time the learning rate will shrink at this rate)
    epochs = 100  # Number of training epochs (maximum number of iterations to run the model through before finding the values corresponding with the minimum error)
    data = None 
    test_dataset = None
    train_dataset = None
    length = 0
    features = [] # Input columns
    target = "" # Output column
    losses = {}  # Store loss values during training

    def __init__(self, train_df, test_df, target, initial_learning_rate=0.01, epochs=100, learning_rate_decay=0.99):
        """
        Initialize the LinearRegression model.

        Parameters:
        - dataset: pandas DataFrame, the dataset to train and test the model.
        - feature_columns: list of str, column names representing features.
        - target_column: str, the column name representing the target variable.
        - initial_learning_rate: float, the initial learning rate for gradient descent.
        - epochs: int, the number of training epochs.
        - learning_rate_decay: float, the factor by which learning rate decays at each epoch.
        """
        # Define class properties
        self.train_dataset = train_df
        self.test_dataset = test_df
        self.target = target
        self.initial_learning_rate = initial_learning_rate
        self.learning_rate_decay = learning_rate_decay
        self.epochs = epochs
        self.length = len(train_df)

    def clean_data(self):

        for df in [self.train_dataset, self.test_dataset]:

            df['date'] = pd.to_datetime(df['date'])
            df['hour'] = df['date'].dt.hour
            df['day_of_week'] = df['date'].dt.dayofweek
            df['day_of_year'] = df['date'].dt.dayofyear
            df['month'] = df['date'].dt.month
            df.drop('date', axis=1, inplace=True)

        self.length = len(self.train_dataset)

        return self.train_dataset, self.test_dataset

    def prepare_data(self, exempt=[]):
        """
        Prepare the datasets for modeling.

        Parameters:
        - targets: The target variables to prepare the data for.

        Returns:
        - train_df, test_df: The prepared training and testing datasets.
        """
        target = self.target
        
        for df in [self.train_dataset, self.test_dataset]:

            # Rolling Sum by Date Cycles
            df[f"{target[:2]}_6houragg"] = df[target].rolling(6).sum()
            df[f"{target[:2]}_12houragg"] = df[target].rolling(12).sum()
            df[f"{target[:2]}_24houragg"] = df[target].rolling(24).sum()

            df.dropna(inplace=True)
            df.reset_index(drop=True, inplace=True)

        self.features = [feature for feature in self.train_dataset.columns if feature != self.target or exempt]
        for feature in self.features:
            self.m.append(0)

        self.length = len(self.train_dataset)

        return self.train_dataset, self.test_dataset, self.features

    def normalize_dataframe(self):
        """
        Normalize all columns in a pandas DataFrame.
        
        Returns:
        - A new DataFrame with normalized values.
        """
        # Calculate the mean and standard deviation for each column using the training dataset
        means = self.train_dataset.mean()
        stds = self.train_dataset.std()

        # Normalize both training and test datasets
        for df in [self.train_dataset, self.test_dataset]:
            # Select non-date columns for normalization
            non_date_columns = [col for col in df.columns if col != 'date']  # Adjust the condition based on your date columns

            # Normalize non-date columns
            df[non_date_columns] = (df[non_date_columns] - means[non_date_columns]) / stds[non_date_columns]

        return self.train_dataset, self.test_dataset

    def calculate_wae_rmse(self):
        """
        Calculate the Weighted Absolute Error (WAE), Root Mean Square Error (RMSE),
        and their respective accuracies for a single target.

        Parameters:
        - reference: The reference dataset containing true values.
        - predictions: The predicted values to compare to.

        Returns:
        - error_metrics: Dictionary containing WAE, RMSE, and their accuracies for the specified target.
        """
        error_metrics = {}
    
        reference = self.test_dataset[self.target]
        predictions = self.predict()

        # Convert DataFrame or Series to NumPy arrays
        y_true = reference.values if isinstance(reference, pd.Series) else reference
        y_pred = predictions.values if isinstance(predictions, pd.Series) else predictions

        # Calculate Weighted Absolute Error (WAE)
        wae = np.sum(np.abs(y_true - y_pred))
        error_metrics['wae'] = wae

        # Calculate Root Mean Square Error (RMSE)
        rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
        error_metrics['rmse'] = rmse

        # Calculate the accuracy for WAE (WAE Accuracy)
        wae_accuracy = 100 - (wae / np.sum(np.abs(y_true)) * 100)
        error_metrics['wae_accuracy'] = wae_accuracy

        # Calculate the accuracy for RMSE (RMSE Accuracy)
        rmse_accuracy = 100 - (rmse / np.std(y_true) * 100)
        error_metrics['rmse_accuracy'] = rmse_accuracy

        self.error_metrics = error_metrics

        return error_metrics

    def loss(self, df):
        """
        Calculate the mean squared error loss of the model on a dataset.

        Parameters:
        - df: pandas DataFrame, the dataset on which to calculate the loss.

        Returns:
        - str: A string representing the total error normalized by the mean target value.
        """
        # Initialize our variables
        total_error = 0
        total_y = 0

        # Iterating through every row of the passed dataframe
        for i in range(len(df)):
            
            # Define our x values as a vector of each feature column at the current row
            x = [df.iloc[i][col] for col in self.features]

            # Grab our expected y value to compare our predicted y value against
            y = df.iloc[i][self.target]

            # Sum the products of each coefficient and corresponding x value 
            total_m = sum(self.m[j] * x[j] for j in range(len(self.features)))
            total_y += (total_m + self.b)

            # Calculate squared difference
            total_error += (y - (total_m + self.b)) ** 2

        # Average values over each row
        total_error /= float(len(df))
        total_y /= float(len(df))

        return total_error, f'Total Error for the Model: {total_error} | Relative Error for the Model: {total_error / abs(total_y)}'

    def gradient_descent(self, m_now, b_now):
        """
        Perform one step of gradient descent to update model coefficients.

        Parameters:
        - m_now: list of float, current model coefficients for features.
        - b_now: float, current model intercept.

        Returns:
        - list of float: Updated model coefficients for features.
        - float: Updated model intercept.
        """
        # Initalize a gradient variable for each feature column
        m_gradients = [0] * (len(self.features))
        b_gradient = 0

        # Iterate through every row
        for i in range(self.length):

            # Define our x values as a vector of each feature column at the current row
            x = [self.train_dataset.iloc[i][col] for col in self.features]

            # Grab our expected y value to compare our predicted y value against
            y = self.train_dataset.iloc[i][self.target]

            # Sum the products of each coefficient and corresponding x value 
            total_m = sum(m_now[j] * x[j] for j in range(len(self.features)))

            # Take the derivative with respect to b
            b_gradient += -(2 / self.length) * (y - (total_m + b_now))

            # Take the derivative with respect to m
            for j in range(len(self.features)):
                m_gradients[j] += -(2 / self.length) * x[j] * (y - (total_m + b_now))

        # Updated the model's coefficients based on the gradient values
        updated_m = [m_now[j] - m_gradients[j] * self.learning_rate for j in range(len(self.features))]
        updated_b = b_now - b_gradient * self.learning_rate

        return updated_m, updated_b

    def train(self, ratio=0.1, max_epochs=None, min_loss_delta=0.01, patience=10):
        """
            Train the Linear Regression model using gradient descent.

            This method iteratively updates the model's coefficients and intercept using gradient descent. It monitors the training process for convergence and stops training based on specified conditions, such as the maximum number of epochs, loss improvement, or patience.

            Parameters:
            - ratio (float): The fraction of the maximum epochs used for checkpointing.
            - max_epochs (int): The maximum number of training epochs.
            - min_loss_delta (float): The minimum change in loss to continue training.
            - patience (int): The number of epochs to tolerate a lack of improvement.

            Returns:
            - None

            Raises:
            - ValueError: If max_epochs is not provided or is less than or equal to 0.

            This method also sets the model's parameters to the values associated with the minimum stored loss during training.

            """
        
        if max_epochs is None:
            raise ValueError("You must provide a value for max_epochs.")
        if max_epochs <= 0:
            raise ValueError("max_epochs must be greater than 0.")
        
        # Initialize variables
        checkpoint = round(max_epochs * ratio)
        learning_rate = self.initial_learning_rate
        prev_loss = float('inf')
        min_loss = float('inf')
        patience_counter = 0
        loss_history = []
        
        for i in range(max_epochs):
            # Save coefficients in case we want to revert back after calculating error
            temp_m = self.m
            temp_b = self.b

            # Determine gradient descent
            self.m, self.b = self.gradient_descent(self.m, self.b)

            # Determine the loss of the updated model 
            loss, string = self.loss(self.train_dataset[:i + 1])
            self.losses[loss] = self.m, self.b

            # If the error loss is increasing by a statistically determined amount add one to the patience counter, which will break the loop when it consecutively reaches the threshold
            if (prev_loss != float('inf')) and ((loss - prev_loss >= prev_loss * 0.5) or (loss >= min_loss)) and (loss < statistics.stdev(self.train_dataset[self.target])):
                patience_counter += 1

            # If the error is moving down, reset the patience counter and update the minimum loss variable
            elif loss < min_loss:
                min_loss = loss
                patience_counter = 0

            # Otherwise the error loss is increasing slightly
            else:
                patience_counter = 0

            # Add the loss to our lost history storage
            loss_history.append(loss)

            # Every predetermined checkpoint we print updates
            if (i % checkpoint == 0) and (i != 0):
                print(f'{time.strftime("%Y-%m-%d %H:%M:%S")} - Epoch {i} - {string}')
                print(self.m, self.b)

            # If the patience counter has surpassed the patience threshold, exit the loop
            if patience_counter >= patience:
                print(f"Training stopped due to lack of improvement for {patience} epochs. Current Error - {loss}")
                break

            # Updated loss and learning rate variables
            prev_loss = loss
            learning_rate *= self.learning_rate_decay
            self.learning_rate = learning_rate

            # Calculate moving average of the loss history
            if i >= patience:
                recent_losses = loss_history[-patience:]
                moving_average = np.mean(recent_losses)
                if moving_average < min_loss_delta:
                    print(f"Training stopped due to a small moving average loss change.")
                    break

        # Set the parameters to the ones with the minimum stored loss
        min_loss = min(self.losses.keys())
        self.m, self.b = self.losses[min_loss]
        print(f'Training Complete. Final Error: {min_loss}')

    def split_datasets(self, frac=0.2):
        """
        Split the dataset into training and testing subsets.

        Parameters:
        - frac: float, the fraction of data to be used for testing.

        Returns:
        - None
        """
        # Split the primary dataset into a test and training dataset based on the passed fraction
        test = self.data.sample(frac=frac)
        train = self.data.sample(frac=1-frac)
        self.train_dataset = train
        self.test_dataset = test

    def predict(self):
        """
        Make predictions using the trained model.

        Parameters:
        - data: pandas DataFrame, the dataset for which to make predictions.

        Returns:
        - list of float: Predicted values based on the model.
        """
        prediction = []

        data = self.test_dataset

        # Iterate through every row
        for i in range(len(data)):

            # Create a vector of all x values at the current column
            x = [data.iloc[i][col] for col in data.columns]

            # Sum the product of all x values and their respective coefficients
            total_m = sum(self.m[j] * x[j] for j in range(len(data.columns)))

            # Calculate the predicted y-value and add it to the output list
            y = total_m + self.b
            prediction.append(y)

        return prediction

    def set_train_dataset(self, dataset):
        """
        Set the training dataset.

        Parameters:
        - dataset: pandas DataFrame, the training dataset.
        """
        self.train_dataset = dataset

    def set_test_dataset(self, dataset):
        """
        Set the testing dataset.

        Parameters:
        - dataset: pandas DataFrame, the testing dataset.
        """
        self.test_dataset = dataset

    def reset_losses(self):
        """
        Reset the list of loss values.
        """
        self.losses = {}

    def list_to_models(self, data, output=None, initial_learning_rate=0.0001, learning_rate_decay=0.999, epochs=500, max_epochs=None, patience=10):
        """
        Generate a dictionary of models with varying feature columns and training each model.

        This method trains multiple Linear Regression models with different sets of feature columns. 
        It iterates through the provided list of feature columns and for each set of features, 
        it trains a Linear Regression model.

        Parameters:
        - data: pandas DataFrame, the dataset to use for training all models.
        - output (dict): A dictionary to store the results of each model. If not provided, 
        a new dictionary will be created.
        - initial_learning_rate (float): The initial learning rate for gradient descent.
        - learning_rate_decay (float): The factor by which learning rate decays at each epoch.
        - epochs (int): The number of training epochs for each model.
        - max_epochs (int): Maximum number of epochs to run for each model.

        Returns:
        - dict: A dictionary containing the results and models for each trained model.
        """

        if output is None:
            output = {}

        # Iterate through the list of feature columns, running a model with an additional feature column each time
        for i in range(1, len(self.features) + 1):
            temp_results = {}
            model_name = 'temp_model_' + str(i)

            # Create a model instance with a subset of feature columns
            model_instance = LinearRegression(data, self.features[:i], 'non-violent', initial_learning_rate, 
            epochs=(epochs + math.floor(i / 2) * 2 * epochs), learning_rate_decay=learning_rate_decay)

            # Set training data as the model's training dataset
            model_instance.set_train_dataset(data)

            # Train the model with a maximum number of epochs and patience
            model_instance.train(max_epochs=epochs, patience=patience)

            # Store the results in a dictionary
            temp_results['results'] = model_instance.model()
            temp_results['model'] = model_instance
            output[model_name] = temp_results

            # Print progress
            print(f"Trained {model_name} with {i} feature columns.")
            print(f"Model: {model_instance.model()}")

        return output
        
    def lasso_regularization(self, alpha):
        """
        Apply L1 (Lasso) regularization to the model.

        Parameters:
        - alpha: float, regularization strength.
        """
        for i in range(len(self.m)):
            if self.m[i] > 0:
                self.m[i] -= alpha
            elif self.m[i] < 0:
                self.m[i] += alpha
        print("L1 (Lasso) regularization applied.")

    def ridge_regularization(self, alpha):
        """
        Apply L2 (Ridge) regularization to the model.

        Parameters:
        - alpha: float, regularization strength.
        """
        for i in range(len(self.m)):
            self.m[i] -= 2 * alpha * self.m[i]
        print("L2 (Ridge) regularization applied.")

    def cross_validation(self, k=5):
        """
        Perform k-fold cross-validation to assess model performance.

        Parameters:
        - k: int, number of folds for cross-validation.

        Returns:
        - dict: Dictionary containing evaluation metrics for each fold.
        """
        # Split data into k folds
        fold_size = len(self.data) // k
        metrics = {'MAE': [], 'RMSE': [], 'R-squared': []}

        for i in range(k):
            # Create train and test datasets for the current fold
            start = i * fold_size
            end = start + fold_size
            test_fold = self.data[start:end]
            train_fold = pd.concat([self.data[:start], self.data[end:]])

            # Train the model on the training fold
            self.set_train_dataset(train_fold)
            self.train()

            # Make predictions on the test fold
            predictions = self.predict(test_fold)

            # Calculate evaluation metrics for this fold
            mae = self.mean_absolute_error(predictions)
            rmse = self.root_mean_squared_error(predictions)
            r_squared = self.r_squared(predictions)

            metrics['MAE'].append(mae)
            metrics['RMSE'].append(rmse)
            metrics['R-squared'].append(r_squared)
            print(f"Cross-validation fold {i+1} completed.")
        
        print("Cross-validation completed.")
        return metrics

    def standardize_features(self):
        """
        Standardize feature columns using StandardScaler.
        """
        scaler = StandardScaler()
        self.data[self.features] = scaler.fit_transform(self.data[self.features])
        print("Features standardized using StandardScaler.")

    def min_max_scaling(self):
        """
        Apply Min-Max scaling to feature columns.
        """
        min_max_scaler = MinMaxScaler()
        self.data[self.features] = min_max_scaler.fit_transform(self.data[self.features])
        print("Min-Max scaling applied to feature columns.")

    def mean_absolute_error(self, predictions):
        """
        Calculate Mean Absolute Error (MAE) for model predictions.

        Parameters:
        - predictions: list of float, predicted values.

        Returns:
        - float: Mean Absolute Error.
        """
        return np.mean(np.abs(predictions - self.test_dataset[self.target]))

    def root_mean_squared_error(self, predictions):
        """
        Calculate Root Mean Squared Error (RMSE) for model predictions.

        Parameters:
        - predictions: list of float, predicted values.

        Returns:
        - float: Root Mean Squared Error.
        """
        return np.sqrt(np.mean((predictions - self.test_dataset[self.target]) ** 2))

    def r_squared(self, predictions):
        """
        Calculate R-squared (coefficient of determination) for model predictions.

        Parameters:
        - predictions: list of float, predicted values.

        Returns:
        - float: R-squared.
        """
        ssr = np.sum((predictions - self.test_dataset[self.target]) ** 2)
        sst = np.sum((self.test_dataset[self.target] - np.mean(self.test_dataset[self.target])) ** 2)
        return 1 - (ssr / sst)

    def hyperparameter_tuning(self, parameter_grid, scoring='neg_mean_squared_error', cv=5):
        """
        Perform hyperparameter tuning using grid search or random search.

        Parameters:
        - parameter_grid: dict, a dictionary of hyperparameter values to search.
        - scoring: str, the scoring metric to optimize.
        - cv: int, the number of cross-validation folds.

        Returns:
        - dict: Dictionary containing the best hyperparameters and their corresponding performance.
        """
        # Create a model instance
        model = LinearRegression(self.data, self.features, self.target)

        # Perform hyperparameter tuning
        grid_search = GridSearchCV(model, parameter_grid, scoring=scoring, cv=cv)
        grid_search.fit(self.data[self.features], self.data[self.target])

        # Get the best hyperparameters and their corresponding performance
        best_hyperparameters = grid_search.best_params_
        best_performance = grid_search.best_score_

        return {'best_hyperparameters': best_hyperparameters, 'best_performance': best_performance}

    def model(self):
        """
        Generate and return the equation of the trained linear regression model.

        Returns:
        - str: The equation of the linear regression model.
        """
        model_parts = [f'({self.m[i]} * {col})' for i, col in enumerate(self.features)]
        model_str = ' + '.join(model_parts)

        if self.b >= 0:
            model = f'y = {model_str} + {self.b}'
        else:
            temp_b = abs(self.b)
            model = f'y = {model_str} - {temp_b}'

        return model

In [4]:
test_df = pd.read_csv('../datasets/representative_samples/test_non-violent_max.csv')
train_df = pd.read_csv('../datasets/representative_samples/train_non-violent_max.csv')

In [5]:
test_model = LinearRegression(test_df=test_df, train_df=train_df, target='non-violent', 
                            initial_learning_rate=0.0001, learning_rate_decay=.999, epochs=10000)

In [6]:
#test_model.clean_data()

In [7]:
test_model.prepare_data()

(                      date  non-violent  violent  train_rides  bike_rides  \
 0      2016-01-02 01:00:00          1.0      0.0      46553.0         0.0   
 1      2016-01-02 02:00:00          0.0      1.0      46553.0         0.0   
 2      2016-01-02 03:00:00          0.0      1.0      46553.0         0.0   
 3      2016-01-02 04:00:00          0.0      0.0      46553.0         2.0   
 4      2016-01-02 05:00:00          0.0      0.0      46553.0         5.0   
 ...                    ...          ...      ...          ...         ...   
 25623  2018-12-31 18:00:00          0.0      0.0      68996.0        12.0   
 25624  2018-12-31 19:00:00          3.0      0.0      68996.0         9.0   
 25625  2018-12-31 20:00:00          1.0      0.0      68996.0         5.0   
 25626  2018-12-31 21:00:00          0.0      0.0      68996.0         8.0   
 25627  2018-12-31 22:00:00          1.0      0.0      68996.0        17.0   
 
        lighting  vacant_buildings  no_6houragg  no_12houragg 

In [8]:
test_model.normalize_dataframe()

TypeError: Could not convert ['2016-01-02 01:00:002016-01-02 02:00:002016-01-02 03:00:002016-01-02 04:00:002016-01-02 05:00:002016-01-02 06:00:002016-01-02 07:00:002016-01-02 08:00:002016-01-02 09:00:002016-01-02 10:00:002016-01-02 11:00:002016-01-02 12:00:002016-01-02 13:00:002016-01-02 14:00:002016-01-02 15:00:002016-01-02 16:00:002016-01-02 17:00:002016-01-02 18:00:002016-01-02 19:00:002016-01-02 20:00:002016-01-02 21:00:002016-01-02 22:00:002016-01-02 23:00:002016-01-03 00:00:002016-01-03 01:00:002016-01-03 02:00:002016-01-03 03:00:002016-01-03 04:00:002016-01-03 05:00:002016-01-03 06:00:002016-01-03 07:00:002016-01-03 08:00:002016-01-03 09:00:002016-01-03 10:00:002016-01-03 11:00:002016-01-03 12:00:002016-01-03 13:00:002016-01-03 14:00:002016-01-03 15:00:002016-01-03 16:00:002016-01-03 17:00:002016-01-03 18:00:002016-01-03 19:00:002016-01-03 20:00:002016-01-03 21:00:002016-01-03 22:00:002016-01-04 00:00:002016-01-04 01:00:002016-01-04 02:00:002016-01-04 03:00:002016-01-04 04:00:00

In [9]:
test_model.model()

'y = (0 * violent) + (0 * train_rides) + (0 * bike_rides) + (0 * lighting) + (0 * vacant_buildings) + (0 * hour) + (0 * day_of_week) + (0 * day_of_year) + (0 * month) + (0 * no_6houragg) + (0 * no_12houragg) + (0 * no_24houragg) + 0'

In [10]:
test_model.train(ratio=0.001, max_epochs=1000, patience=100)

2023-11-13 17:55:57 - Epoch 1 - Total Error for the Model: 1.324167699720764e+29 | Relative Error for the Model: 363891151269272.8
[-11933.847289968304, -7816704091.595448, -9358206.50562035, -1006047.4856945632, -1828672.8568118769, -641923.5772617975, -141431.26997559203, -10581908.083370669, -376741.9985387002, -344321.08308865665, -682873.8135121308, -1345196.7171643195] -57267.10259472624
2023-11-13 17:56:04 - Epoch 2 - Total Error for the Model: 1.4003642935268783e+42 | Relative Error for the Model: 1.1833698887190252e+21
[38808736290.07221, 2.5419832933913216e+16, 30432761655741.902, 3271655053169.6113, 5946823196774.764, 2087528173062.7483, 459932845701.0048, 34412240298501.03, 1225160547303.0415, 1119728372445.2937, 2220698127126.426, 4374564404846.9873] 186231962196.30765
2023-11-13 17:56:12 - Epoch 3 - Total Error for the Model: 1.4779850988651092e+55 | Relative Error for the Model: 3.8444571773725216e+27
[-1.2607936406752789e+17, -8.258234298216388e+22, -9.886802826281948e+

KeyboardInterrupt: 

In [ ]:
test_model.calculate_wae_rmse()

In [ ]:
model_losses = models['test_model_1']['model'].losses
Q1 = np.percentile(model_losses, 25)
Q3 = np.percentile(model_losses, 75)
IQR = Q3 - Q1

# Define a lower and upper bound
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Remove outliers based on the bounds
outliers_removed = [x for x in model_losses if lower_bound <= x <= upper_bound]

plt.plot(range(len(outliers_removed)), outliers_removed)